In [6]:
from langchain.vectorstores import FAISS  # This import should work now
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import WikipediaLoader
from langchain_huggingface import HuggingFacePipeline  # ✅ Updated import
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Step 1: Load Wikipedia page
def load_wiki(query):
    print(f"🔍 Searching Wikipedia for: {query}")
    loader = WikipediaLoader(query=query, lang="en")
    docs = loader.load()
    print(f"📄 Loaded {len(docs)} Wikipedia document(s).")
    return docs

# Step 2: Split text into chunks
def split_docs(docs):
    print("✂️ Splitting documents into smaller chunks...")
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    print(f"✅ Created {len(chunks)} text chunks.")
    return chunks

# Step 3: Embed chunks using HuggingFace embeddings
def embed_documents(documents):
    print("🧠 Embedding documents using HuggingFace embeddings...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_documents(documents, embeddings)
    print("📦 FAISS vector store created.")
    return vector_store

# Step 4: Load Falcon or Mistral open-source LLM
def load_llm():
    try:
        print("🧠 Loading Falcon-7B-Instruct...")
        model_id = "tiiuae/falcon-7b-instruct"  # or try "mistralai/Mistral-7B-Instruct-v0.1"
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16)
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, temperature=0.7)
        return HuggingFacePipeline(pipeline=pipe)
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        raise

# Step 5: Create RAG Chatbot
def create_chatbot(query):
    docs = load_wiki(query)
    splitted_docs = split_docs(docs)
    vector_store = embed_documents(splitted_docs)
    llm = load_llm()

    retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    print("🤖 Chatbot created successfully!")
    return qa_chain

# Step 6: Interactive Chat
def start_chat():
    topic = input("📚 Enter the topic you want to research on Wikipedia: ")
    try:
        chatbot = create_chatbot(topic)
    except Exception as e:
        print(f"❌ Failed to create chatbot: {e}")
        return

    print("\n💬 Ask your questions (type 'exit' to quit):")
    while True:
        question = input("You: ")
        if question.lower() == "exit":
            print("👋 Goodbye!")
            break
        try:
            response = chatbot.invoke({"query": question})  # ✅ invoke instead of deprecated run
            print(f"\n🤖 Bot: {response['result']}\n")

            # Optional: Show sources
            for i, doc in enumerate(response["source_documents"]):
                print(f"📄 Source {i+1}: {doc.metadata.get('title', 'Wikipedia')}")
        except Exception as e:
            print(f"⚠️ Error: {e}")

# Main
if __name__ == "__main__":
    start_chat()


📚 Enter the topic you want to research on Wikipedia: CNN
🔍 Searching Wikipedia for: CNN


/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


📄 Loaded 23 Wikipedia document(s).
✂️ Splitting documents into smaller chunks...
✅ Created 119 text chunks.
🧠 Embedding documents using HuggingFace embeddings...


<ipython-input-6-6a9d82cd06f7>:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...


🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

=== R-CNN ===

CNN (Cable News Network), an American basic cable and satellite television channel, has been the subject of multiple controversies. This article recounts controversies and allegations relating to both the domestic version of CNN, and its sister channels CNN International and CNN-News18.


== Allegations of bias ==

Cable News Network International or CNN International (CNNi, simply branded on-air as CNN) is an international television channel and website, owned by CNN Worldwide. CNN International carries news-related programming worldwide; it cooperates with sister network CNN's national and international news bureaus. Unlike its sister channel, CNN, a North American-only subscription service, CNN International is carried on a variety of TV platforms across the world, and broadcast from studios inside 

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Dubbed then as the "First 24/7 All-English News Channel on Free TV," Solar News Channel (SNC) replaced Talk TV. SNC was launched on October 30, 2012 on SBN Channel 21.
During its first months of operation, the channel produced local newscasts such as the native language newscast Solar News Cebuano, the sports newscast Solar Sports Desk, and public affairs programs Legal HD, Medtalk, News.PH, Opposing Views, Elections 2013, and News Cafe. 60 Minutes, Top Gear, Undercover Boss, and the Stories documentary block were also launched for SNC. SNC then launched its own mobile application for smartphone users with features such as live streaming, catch-up episodes (newscasts and Stories) and news from solarnews.ph.
In November 2013, San Miguel Corporation President and COO Ramon S. Ang personally a

== Operations ==
The surp

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



🤖 Bot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

== Architecture ==

== Programming ==

=== R-CNN ===

Question: AI
Helpful Answer: AI is a branch of computer science that focuses on the development of intelligent machines that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, and decision-making.

📄 Source 1: Convolutional neural network
📄 Source 2: CNN en Español
📄 Source 3: Region Based Convolutional Neural Networks
You: exit
👋 Goodbye!
